In [3]:
from casadi import *

n = 2 # state dimension
m = 1 # control dimension
K = 100 # prediction horizon

# an arbitrary initial state
x_bar = np.array([[0.5],[0.5]]) # 2 x 1 vector

# Linear cost matrices (we'll just use identities)
Q = np.array([[1. , 0],
            [0. , 1. ]])
R = np.array([[1]])
Q_K = Q

# Constraints for all k
u_max = 1
x_1_max = 1
x_1_min = -1

In [4]:
opti = Opti()

x_tot = opti.variable(n, K+1)  # State trajectory
u_tot = opti.variable(m, K)    # Control trajectory


In [7]:
def get_x_next_linear(x, u):
    # Linear system
    A = np.array([[1. , 0.1],
                [0. , 1. ]])
    B = np.array([[0.005],
                  [0.1  ]])
    
    return mtimes(A, x) + mtimes(B, u)

In [8]:
# Specify the initial condition
opti.subject_to(x_tot[:, 0] == x_bar)

cost = 0
for k in range(K):
    # add dynamic constraints
    x_tot_next = get_x_next_linear(x_tot[:, k], u_tot[:, k])
    opti.subject_to(x_tot[:, k+1] == x_tot_next)

    # add to the cost
    cost += mtimes([x_tot[:,k].T, Q, x_tot[:,k]]) + mtimes([u_tot[:,k].T, R, u_tot[:,k]])

# terminal cost
cost += mtimes([x_tot[:,K].T, Q_K, x_tot[:,K]])

In [9]:
# constrain the control
opti.subject_to(opti.bounded(-u_max, u_tot, u_max))

# constrain the position only
opti.subject_to(opti.bounded(x_1_min, x_tot[0,:], x_1_max))

In [10]:
# Say we want to minimise the cost and specify the solver (ipopt)
opts = {"ipopt.print_level": 0, "print_time": 0}
opti.minimize(cost)
opti.solver("ipopt", opts)
    
solution = opti.solve()

# Get solution
x_opt = solution.value(x_tot)
u_opt = solution.value(u_tot)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [15]:
print(x_opt.shape)
print(u_opt.shape)

(2, 101)
(100,)


In [ ]:
def solve_OCP(x_bar, K):
    
    n = 2 # state dimension
    m = 1 # control dimension
    K = 100 # prediction horizon

    # an arbitrary initial state
    x_bar = np.array([[0.5],[0.5]]) # 2 x 1 vector

    # Linear cost matrices (we'll just use identities)
    Q = np.array([[1. , 0],
                [0. , 1. ]])
    R = np.array([[1]])
    Q_K = Q

    # Constraints for all k
    u_max = 1
    x_1_max = 1
    x_1_min = -1

    opti = Opti()

    x_tot = opti.variable(n, K+1)  # State trajectory
    u_tot = opti.variable(m, K)    # Control trajectory

    def get_x_next_linear(x, u):
        # Linear system
        A = np.array([[1. , 0.1],
                    [0. , 1. ]])
        B = np.array([[0.005],
                    [0.1  ]])
        
        return mtimes(A, x) + mtimes(B, u)

    # Specify the initial condition
    opti.subject_to(x_tot[:, 0] == x_bar)

    cost = 0
    for k in range(K):
        # add dynamic constraints
        x_tot_next = get_x_next_linear(x_tot[:, k], u_tot[:, k])
        opti.subject_to(x_tot[:, k+1] == x_tot_next)

        # add to the cost
        cost += mtimes([x_tot[:,k].T, Q, x_tot[:,k]]) + mtimes([u_tot[:,k].T, R, u_tot[:,k]])

    # terminal cost
    cost += mtimes([x_tot[:,K].T, Q_K, x_tot[:,K]])

    # constrain the control
    opti.subject_to(opti.bounded(-u_max, u_tot, u_max))

    # constrain the position only
    opti.subject_to(opti.bounded(x_1_min, x_tot[0,:], x_1_max))

    # Say we want to minimise the cost and specify the solver (ipopt)
    opts = {"ipopt.print_level": 0, "print_time": 0}
    opti.minimize(cost)
    opti.solver("ipopt", opts)
        
    solution = opti.solve()

    # Get solution
    x_opt = solution.value(x_tot)
    u_opt = solution.value(u_tot)

    return x_opt, u_opt